In [51]:
import pandas as pd
import os
filePath = "AlltheMatrix/"
single_version_file_path = filePath + 'modified_single_version-ck-oo.csv'
changed_file_path = filePath + 'modified_change_metrics.csv'
bug_matrix_path = filePath + 'modified_bug-metrics.csv' 
# complexity_code_change_path = 



filePathNowusing = changed_file_path

def load_one_version_method_matrix(matrix_path):
    print(matrix_path)
    return pd.read_csv(matrix_path)

changed_matrix = load_one_version_method_matrix(filePathNowusing)
print(f'lens: {len(changed_matrix.columns.values)}')

startindex = 0
endindex = 0
for i, column in enumerate(changed_matrix.columns.values):
    if(column == 'classname'):
        startindex = i
    elif (column == 'bugs'):
        endindex = i
print(f'start:{startindex},end:{endindex}') 
target = changed_matrix['bugs'].values
data = changed_matrix.values[:,startindex+1:endindex]
print(target.shape)
print(data.shape)

#shuffleIndex
import numpy as np
shuffle_index = np.random.permutation(997)
target, data = target[shuffle_index], data[shuffle_index]

#shuffleIndex
import numpy as np
shuffle_index = np.random.permutation(997)
target, data = target[shuffle_index], data[shuffle_index]

AlltheMatrix/modified_change_metrics.csv
lens: 23
start:1,end:17
(997,)
(997, 15)


In [218]:
from itertools import combinations
columnlist = np.arange(0,endindex-startindex-1,1)
print(columnlist)
columncombation = list(combinations(columnlist, 5))
type(columncombation[0])
len(columncombation)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]


3003

# Random Forest Classifier

In [111]:
print(X_train),columncombation[0],len(X_train.T)

[[47 7 0 ... 4.29787 367.0 154.785]
 [20 4 0 ... 0.45 243.0 94.4892]
 [15 4 0 ... 17.7333 218.857 13.0702]
 ...
 [4 3 0 ... 0.0 198.143 45.2857]
 [124 10 0 ... 4.29839 329.0 99.3253]
 [7 3 0 ... 0.0 220.143 45.0952]]


(None, (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), 15)

In [210]:
def get_selected_rows(columncombation,X_data):
    flag = False
    for i in columncombation:
        if(flag == False):
            c = X_data[:,i].T
            flag = True
        else:
            c = np.vstack((c,X_data[:,i].T))
    X_train_selected = c.T 
    return X_train_selected
def predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs):
    n = 0
    bugrowcount = 0
    for i in range(0,len(Y_test_predict)):
        if(Y_test_bugs[i]!=False):
            bugrowcount += 1
            if(Y_test_predict[i]==Y_test_bugs[i]):
                n=n+1
#    print(n/bugrowcount,n,bugrowcount)
    return n/bugrowcount,n

In [219]:
#split the train set and the test set
X_train = data[:700]
X_test = data[700:]
Y_train = target[:700]
Y_test = target[700:]
print(X_train,Y_train.shape)
#train a binary classifier
Y_train_bugs = (Y_train > 0)
Y_test_bugs = (Y_test > 0 )
from sklearn.ensemble import RandomForestClassifier
#test the result: cross-verification
from sklearn.model_selection import cross_val_score
cent = 0
for columncombation_j in columncombation:
    X_train_selected = get_selected_rows(columncombation_j,X_train)
    X_test_selected = get_selected_rows(columncombation_j,X_test)


    forest_clf = RandomForestClassifier(n_estimators = 10)
    forest_clf.fit(X_train_selected,Y_train_bugs)

    cross_result = cross_val_score(forest_clf, X_train_selected, Y_train_bugs, cv = 10, scoring = 'accuracy')
    Y_forest_predict = forest_clf.predict(X_train_selected)

    from sklearn.metrics import precision_score, recall_score
    forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
    forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
    print(f'Forest_accurancy is {forst_precision}')
    print(f'Forest_recall is {forst_recall}')

    print(columncombation_j)
#    print(X_train_selected)
    Y_test_predict = forest_clf.predict(X_test_selected)
    
    findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
    print(f'fingdingrate = {findingRate}')

    import csv
    with open('Rows_selected_5_result.csv','a+') as f:
        csv_write = csv.writer(f)
        data_row = [columncombation_j,forst_precision,forst_recall,findingRate,cross_result]
        csv_write.writerow(data_row)
    print(f'#########finishing:{cent/len(columncombation)}')
    cent+=1
print('------------------------------------------ \
----------------------------------------\n')


[[18 4 0 ... 0.44444399999999995 367.0 55.9708]
 [38 3 0 ... 0.605263 248.143 67.559]
 [26 8 0 ... 0.923077 322.714 76.2345]
 ...
 [24 8 0 ... 1.5 333.14300000000003 66.5175]
 [14 5 0 ... 2.42857 159.857 46.9885]
 [22 4 0 ... 22.3182 226.0 5.52844]] (700,)
Forest_accurancy is 1.0
Forest_recall is 0.9121621621621622
(0, 1, 2, 3, 4)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.0
Forest_accurancy is 0.9854014598540146
Forest_recall is 0.9121621621621622
(0, 1, 2, 3, 5)
fingdingrate = (0.41379310344827586, 24)
#########finishing:0.000333000333000333
Forest_accurancy is 0.9772727272727273
Forest_recall is 0.8716216216216216
(0, 1, 2, 3, 6)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.000666000666000666
Forest_accurancy is 1.0
Forest_recall is 0.8851351351351351
(0, 1, 2, 3, 7)
fingdingrate = (0.4827586206896552, 28)
#########finishing:0.000999000999000999
Forest_accurancy is 0.9777777777777777
Forest_recall is 0.8918918918918919
(0, 1, 2, 3, 8)
fingdingrate

Forest_accurancy is 0.9710144927536232
Forest_recall is 0.9054054054054054
(0, 1, 2, 8, 13)
fingdingrate = (0.5172413793103449, 30)
#########finishing:0.016317016317016316
Forest_accurancy is 1.0
Forest_recall is 0.8581081081081081
(0, 1, 2, 8, 14)
fingdingrate = (0.5344827586206896, 31)
#########finishing:0.016650016650016652
Forest_accurancy is 0.9923664122137404
Forest_recall is 0.8783783783783784
(0, 1, 2, 9, 10)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.016983016983016984
Forest_accurancy is 0.9924812030075187
Forest_recall is 0.8918918918918919
(0, 1, 2, 9, 11)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.017316017316017316
Forest_accurancy is 0.9854014598540146
Forest_recall is 0.9121621621621622
(0, 1, 2, 9, 12)
fingdingrate = (0.3793103448275862, 22)
#########finishing:0.017649017649017648
Forest_accurancy is 0.9923664122137404
Forest_recall is 0.8783783783783784
(0, 1, 2, 9, 13)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.

Forest_accurancy is 0.9841269841269841
Forest_recall is 0.8378378378378378
(0, 1, 3, 7, 14)
fingdingrate = (0.39655172413793105, 23)
#########finishing:0.03296703296703297
Forest_accurancy is 0.9924242424242424
Forest_recall is 0.8851351351351351
(0, 1, 3, 8, 9)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.033300033300033303
Forest_accurancy is 0.9925925925925926
Forest_recall is 0.9054054054054054
(0, 1, 3, 8, 10)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.03363303363303363
Forest_accurancy is 1.0
Forest_recall is 0.8986486486486487
(0, 1, 3, 8, 11)
fingdingrate = (0.39655172413793105, 23)
#########finishing:0.03396603396603397
Forest_accurancy is 0.9925373134328358
Forest_recall is 0.8986486486486487
(0, 1, 3, 8, 12)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.034299034299034296
Forest_accurancy is 0.9858156028368794
Forest_recall is 0.9391891891891891
(0, 1, 3, 8, 13)
fingdingrate = (0.39655172413793105, 23)
#########finishing:0.

Forest_accurancy is 0.9784172661870504
Forest_recall is 0.918918918918919
(0, 1, 4, 8, 13)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.049617049617049616
Forest_accurancy is 0.9847328244274809
Forest_recall is 0.8716216216216216
(0, 1, 4, 8, 14)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.04995004995004995
Forest_accurancy is 1.0
Forest_recall is 0.8716216216216216
(0, 1, 4, 9, 10)
fingdingrate = (0.4827586206896552, 28)
#########finishing:0.05028305028305028
Forest_accurancy is 1.0
Forest_recall is 0.8918918918918919
(0, 1, 4, 9, 11)
fingdingrate = (0.4827586206896552, 28)
#########finishing:0.050616050616050616
Forest_accurancy is 1.0
Forest_recall is 0.8716216216216216
(0, 1, 4, 9, 12)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.05094905094905095
Forest_accurancy is 0.9862068965517241
Forest_recall is 0.9662162162162162
(0, 1, 4, 9, 13)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.05128205128205128
Forest_accurancy

Forest_accurancy is 0.9777777777777777
Forest_recall is 0.8918918918918919
(0, 1, 5, 12, 13)
fingdingrate = (0.4827586206896552, 28)
#########finishing:0.06626706626706627
Forest_accurancy is 0.9925373134328358
Forest_recall is 0.8986486486486487
(0, 1, 5, 12, 14)
fingdingrate = (0.5172413793103449, 30)
#########finishing:0.06660006660006661
Forest_accurancy is 0.9927536231884058
Forest_recall is 0.9256756756756757
(0, 1, 5, 13, 14)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.06693306693306693
Forest_accurancy is 1.0
Forest_recall is 0.9121621621621622
(0, 1, 6, 7, 8)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.06726606726606726
Forest_accurancy is 0.9925373134328358
Forest_recall is 0.8986486486486487
(0, 1, 6, 7, 9)
fingdingrate = (0.5, 29)
#########finishing:0.0675990675990676
Forest_accurancy is 1.0
Forest_recall is 0.8581081081081081
(0, 1, 6, 7, 10)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.06793206793206794
Forest_accurancy

...

Forest_accurancy is 1.0
Forest_recall is 0.8918918918918919
(5, 6, 10, 12, 14)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.9327339327339327
Forest_accurancy is 0.9856115107913669
Forest_recall is 0.9256756756756757
(5, 6, 10, 13, 14)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.9330669330669331
Forest_accurancy is 0.9716312056737588
Forest_recall is 0.9256756756756757
(5, 6, 11, 12, 13)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.9333999333999334
Forest_accurancy is 1.0
Forest_recall is 0.8851351351351351
(5, 6, 11, 12, 14)
fingdingrate = (0.3793103448275862, 22)
#########finishing:0.9337329337329338
Forest_accurancy is 0.9782608695652174
Forest_recall is 0.9121621621621622
(5, 6, 11, 13, 14)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.9340659340659341
Forest_accurancy is 0.9784172661870504
Forest_recall is 0.918918918918919
(5, 6, 12, 13, 14)
fingdingrate = (0.39655172413793105, 23)
#########finishing:0.9343989343989

Forest_accurancy is 0.9923664122137404
Forest_recall is 0.8783783783783784
(5, 8, 9, 13, 14)
fingdingrate = (0.3793103448275862, 22)
#########finishing:0.9493839493839494
Forest_accurancy is 1.0
Forest_recall is 0.9121621621621622
(5, 8, 10, 11, 12)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.9497169497169498
Forest_accurancy is 0.9861111111111112
Forest_recall is 0.9594594594594594
(5, 8, 10, 11, 13)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.9500499500499501
Forest_accurancy is 0.9920634920634921
Forest_recall is 0.8445945945945946
(5, 8, 10, 11, 14)
fingdingrate = (0.39655172413793105, 23)
#########finishing:0.9503829503829504
Forest_accurancy is 0.9855072463768116
Forest_recall is 0.918918918918919
(5, 8, 10, 12, 13)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.9507159507159507
Forest_accurancy is 0.9921875
Forest_recall is 0.8581081081081081
(5, 8, 10, 12, 14)
fingdingrate = (0.39655172413793105, 23)
#########finishing:0.95104895

Forest_accurancy is 0.9925373134328358
Forest_recall is 0.8986486486486487
(6, 7, 9, 13, 14)
fingdingrate = (0.43103448275862066, 25)
#########finishing:0.9660339660339661
Forest_accurancy is 1.0
Forest_recall is 0.8716216216216216
(6, 7, 10, 11, 12)
fingdingrate = (0.41379310344827586, 24)
#########finishing:0.9663669663669664
Forest_accurancy is 0.9850746268656716
Forest_recall is 0.8918918918918919
(6, 7, 10, 11, 13)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.9666999666999667
Forest_accurancy is 1.0
Forest_recall is 0.8783783783783784
(6, 7, 10, 11, 14)
fingdingrate = (0.4827586206896552, 28)
#########finishing:0.967032967032967
Forest_accurancy is 0.9710144927536232
Forest_recall is 0.9054054054054054
(6, 7, 10, 12, 13)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.9673659673659674
Forest_accurancy is 0.9779411764705882
Forest_recall is 0.8986486486486487
(6, 7, 10, 12, 14)
fingdingrate = (0.41379310344827586, 24)
#########finishing:0.967698967698

Forest_accurancy is 0.9924242424242424
Forest_recall is 0.8851351351351351
(7, 8, 9, 11, 12)
fingdingrate = (0.3793103448275862, 22)
#########finishing:0.9826839826839827
Forest_accurancy is 0.9858156028368794
Forest_recall is 0.9391891891891891
(7, 8, 9, 11, 13)
fingdingrate = (0.5172413793103449, 30)
#########finishing:0.983016983016983
Forest_accurancy is 1.0
Forest_recall is 0.8986486486486487
(7, 8, 9, 11, 14)
fingdingrate = (0.41379310344827586, 24)
#########finishing:0.9833499833499834
Forest_accurancy is 0.9788732394366197
Forest_recall is 0.9391891891891891
(7, 8, 9, 12, 13)
fingdingrate = (0.46551724137931033, 27)
#########finishing:0.9836829836829837
Forest_accurancy is 0.9921259842519685
Forest_recall is 0.8513513513513513
(7, 8, 9, 12, 14)
fingdingrate = (0.41379310344827586, 24)
#########finishing:0.984015984015984
Forest_accurancy is 1.0
Forest_recall is 0.8918918918918919
(7, 8, 9, 13, 14)
fingdingrate = (0.4482758620689655, 26)
#########finishing:0.9843489843489843
For

Forest_accurancy is 0.9790209790209791
Forest_recall is 0.9459459459459459
(9, 11, 12, 13, 14)
fingdingrate = (0.3448275862068966, 20)
#########finishing:0.9993339993339994
Forest_accurancy is 0.9855072463768116
Forest_recall is 0.918918918918919
(10, 11, 12, 13, 14)
fingdingrate = (0.3620689655172414, 21)
#########finishing:0.9996669996669997
------------------------------------------ ----------------------------------------



In [38]:
columnlist = changed_matrix.columns.values
print(columnlist)
predicted_bugs_matrix = pd.DataFrame(columns=columnlist)
predicted_bugs_matrix = predicted_bugs_matrix.append(changed_matrix.iloc[0], ignore_index=True)
print(predicted_bugs_matrix)
predicted_bugs_matrix = pd.DataFrame(columns=columnlist)
for i,Y_one in enumerate(Y_forest_predict):
    if(Y_one == False):
        predicted_bugs_matrix = predicted_bugs_matrix.append(changed_matrix.iloc[i], ignore_index=True)
predicted_bugs_matrix

['Unnamed: 0' 'classname' 'numberOfVersionsUntil' 'numberOfFixesUntil'
 'numberOfRefactoringsUntil' 'numberOfAuthorsUntil' 'linesAddedUntil'
 'maxLinesAddedUntil' 'avgLinesAddedUntil' 'linesRemovedUntil'
 'maxLinesRemovedUntil' 'avgLinesRemovedUntil' 'codeChurnUntil'
 'maxCodeChurnUntil' 'avgCodeChurnUntil' 'ageWithRespectTo'
 'weightedAgeWithRespectTo' 'bugs' 'nonTrivialBugs' 'majorBugs'
 'criticalBugs' 'highPriorityBugs' 'Unnamed: 22']
  Unnamed: 0                                          classname  \
0          0  org::eclipse::jdt::internal::core::search::ind...   

  numberOfVersionsUntil numberOfFixesUntil numberOfRefactoringsUntil  \
0                    65                  4                         0   

  numberOfAuthorsUntil  linesAddedUntil  maxLinesAddedUntil  \
0                    8            608.0               158.0   

   avgLinesAddedUntil  linesRemovedUntil  ...  maxCodeChurnUntil  \
0             9.35385              566.0  ...               15.0   

   avgCodeChur

,Unnamed: 0,classname,numberOfVersionsUntil,numberOfFixesUntil,numberOfRefactoringsUntil,numberOfAuthorsUntil,linesAddedUntil,maxLinesAddedUntil,avgLinesAddedUntil,linesRemovedUntil,...,maxCodeChurnUntil,avgCodeChurnUntil,ageWithRespectTo,weightedAgeWithRespectTo,bugs,nonTrivialBugs,majorBugs,criticalBugs,highPriorityBugs,Unnamed: 22
0,5,org::eclipse::jdt::internal::formatter::Location,16,5,0,5,37.0,6.0,2.31250,17.0,...,4.0,1.250000,264.85700,48.82240,0,0,0,0,0,
1,21,org::eclipse::jdt::internal::core::util::Signa...,6,3,0,3,104.0,49.0,17.33330,6.0,...,49.0,16.333300,228.85700,14.81730,0,0,0,0,0,
2,22,org::eclipse::jdt::internal::core::BinaryMember,43,6,0,6,409.0,105.0,9.51163,317.0,...,79.0,2.139530,367.00000,133.86100,1,0,0,0,0,
3,30,org::eclipse::jdt::internal::compiler::classfm...,2,1,0,2,1.0,1.0,0.50000,1.0,...,0.0,0.000000,127.00000,69.28570,0,0,0,0,0,
4,32,org::eclipse::jdt::internal::compiler::ast::Ar...,47,7,0,5,320.0,52.0,6.80851,262.0,...,20.0,1.234040,367.00000,92.41380,0,0,0,0,0,
5,36,org::eclipse::jdt::core::dom::SuperFieldAccess,39,14,0,7,465.0,167.0,11.92310,264.0,...,74.0,5.153850,333.14300,70.56900,0,0,0,0,0,
6,38,org::eclipse::jdt::internal::compiler::lookup:...,89,9,0,9,2001.0,399.0,22.48310,1889.0,...,117.0,1.258430,367.00000,118.10300,1,1,0,0,0,
7,42,org::eclipse::jdt::internal::compiler::ast::Fo...,86,11,0,8,1629.0,335.0,18.94190,1535.0,...,65.0,1.093020,367.00000,76.47740,0,0,0,0,0,
8,49,org::eclipse::jdt::internal::compiler::ast::No...,18,9,1,6,211.0,84.0,11.72220,53.0,...,84.0,8.777780,222.71400,21.60870,0,0,0,0,0,
9,55,org::eclipse::jdt::core::dom::MethodDeclaration,73,16,0,8,2096.0,439.0,28.71230,1160.0,...,282.0,12.821900,333.14300,84.08830,0,0,0,0,0,


In [187]:
columncombation_j = (2,3,4,5,6,7,14)
X_train_selected = get_selected_rows(columncombation_j,X_train)
X_test_selected = get_selected_rows(columncombation_j,X_test)


forest_clf = RandomForestClassifier(n_estimators = 10)
forest_clf.fit(X_train_selected,Y_train_bugs)

cross_result = cross_val_score(forest_clf, X_train_selected, Y_train_bugs, cv = 10, scoring = 'accuracy')
Y_forest_predict = forest_clf.predict(X_train_selected)

from sklearn.metrics import precision_score, recall_score
forst_precision = precision_score(Y_train_bugs, Y_forest_predict)
forst_recall = recall_score(Y_train_bugs, Y_forest_predict)
print(f'Forest_accurancy is {forst_precision}')
print(f'Forest_recall is {forst_recall}')

print(columncombation_j)
#    print(X_train_selected)
print(len(X_test_selected[0]))
Y_test_predict = forest_clf.predict(X_test_selected)

findingRate = predicted_bugNum_vs_true_bugNum(Y_test_predict,Y_test_bugs)
print(f'fingdingrate = {findingRate}')

Forest_accurancy is 1.0
Forest_recall is 0.8513513513513513
(2, 3, 4, 5, 6, 7, 14)
7
fingdingrate = 0.4482758620689655


In [178]:
predicted_bugs_matrix.to_csv("Predicted"+filePathNowusing)

# use alamo to regress the buggy code

### have troubles when using the alamo